# Churn — Preprocesamiento y Feature Engineering

**Objetivo:** construir la variable objetivo (*churn*) y generar un dataset a nivel **cliente** (recencia, frecuencia, gasto, reseñas y pagos) para usarlo en el modelado de clasificación.


## Carga de datasets base

Se cargan las tablas principales necesarias para comenzar el análisis (órdenes y clientes).

In [68]:
import pandas as pd
import numpy as np
orders = pd.read_csv("../data/olist_orders_dataset.csv")
customers = pd.read_csv("../data/olist_customers_dataset.csv")


## Preparación de fechas y última compra

Se convierten columnas de fecha a `datetime` y se calcula la **última compra** por cliente para poder medir recencia.

In [69]:
# Asegurar que la fecha esté en formato datetime
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])

# Última compra por cliente
last_purchase = orders.groupby("customer_id")["order_purchase_timestamp"].max().reset_index()
last_purchase.columns = ["customer_id", "last_purchase_date"]

last_purchase.head()


,customer_id,last_purchase_date
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17


## Recencia: días desde la última compra

Se define una fecha de referencia (máxima fecha disponible) y se calcula cuántos días pasaron desde la última compra de cada cliente.

In [70]:
max_date = orders["order_purchase_timestamp"].max()
max_date


Timestamp('2018-10-17 17:30:18')

In [71]:
last_purchase["days_since_last_purchase"] = (
    max_date - last_purchase["last_purchase_date"]
).dt.days

last_purchase.head()


,customer_id,last_purchase_date,days_since_last_purchase
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,337
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,458
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,596
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,427
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,198


## Variable objetivo (churn)

**Definición:** un cliente se considera *churn* si pasaron **más de 180 días** desde su última compra.
> Este umbral es una decisión de negocio; puede ajustarse según el contexto.


In [72]:
# Cliente es "desertor" si pasaron más de 180 días desde su última compra
last_purchase["churn"] = (last_purchase["days_since_last_purchase"] > 180).astype(int)

last_purchase.head()


,customer_id,last_purchase_date,days_since_last_purchase,churn
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,337,1
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,458,1
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,596,1
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,427,1
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,198,1


## Dataset base a nivel cliente

Se integra la información de churn con la tabla de clientes para construir el dataset base por `customer_id`.

In [73]:
customers_churn = customers.merge(last_purchase, on="customer_id", how="left")

customers_churn.head()


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,last_purchase_date,days_since_last_purchase,churn
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,2017-05-16 15:05:35,519,1
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,2018-01-12 20:48:24,277,1
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,2018-05-19 16:07:45,151,0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,2018-03-13 16:06:38,218,1
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,2018-07-29 09:51:30,80,0


## Carga de tablas auxiliares (items, reviews, payments)

Se cargan tablas necesarias para construir features adicionales (frecuencia, gasto, reseñas y pagos).

In [74]:
order_items = pd.read_csv("../data/olist_order_items_dataset.csv")
reviews = pd.read_csv("../data/olist_order_reviews_dataset.csv")
payments = pd.read_csv("../data/olist_order_payments_dataset.csv")


In [75]:
order_reviews = pd.read_csv("../data/olist_order_reviews_dataset.csv")
order_payments = pd.read_csv("../data/olist_order_payments_dataset.csv")
print("reviews:", order_reviews.shape, "| payments:", order_payments.shape)


reviews: (99224, 7) | payments: (103886, 5)


## Feature: frecuencia de compra

Cantidad de órdenes por cliente (proxy de frecuencia).

In [76]:
purchase_frequency = (
    orders.groupby("customer_id")["order_id"]
          .nunique()
          .reset_index()
)

purchase_frequency.columns = ["customer_id", "purchase_count"]
purchase_frequency.head()


,customer_id,purchase_count
0,00012a2ce6f8dcda20d059ce98491703,1
1,000161a058600d5901f007fab4c27140,1
2,0001fd6190edaaf884bcaf3d49edf079,1
3,0002414f95344307404f0ace7a26f1d5,1
4,000379cdec625522490c315e70c7a9fb,1


## Feature: ticket promedio / gasto

Se combina `orders` con `order_items` para estimar el gasto por cliente y calcular el ticket promedio.

In [77]:
# Unimos orders con order_items para saber cuánto gastó cada cliente
orders_items = orders.merge(order_items, on="order_id", how="left")

total_spent = (
    orders_items.groupby("customer_id")["price"]
                .sum()
                .reset_index()
)

total_spent.columns = ["customer_id", "total_spent"]
total_spent.head()


,customer_id,total_spent
0,00012a2ce6f8dcda20d059ce98491703,89.80
1,000161a058600d5901f007fab4c27140,54.90
2,0001fd6190edaaf884bcaf3d49edf079,179.99
3,0002414f95344307404f0ace7a26f1d5,149.90
4,000379cdec625522490c315e70c7a9fb,93.00


In [78]:
avg_ticket = (
    orders_items.groupby("customer_id")["price"]
                .mean()
                .reset_index()
)

avg_ticket.columns = ["customer_id", "avg_ticket"]
avg_ticket.head()


,customer_id,avg_ticket
0,00012a2ce6f8dcda20d059ce98491703,89.80
1,000161a058600d5901f007fab4c27140,54.90
2,0001fd6190edaaf884bcaf3d49edf079,179.99
3,0002414f95344307404f0ace7a26f1d5,149.90
4,000379cdec625522490c315e70c7a9fb,93.00


## Feature: reseñas

Se agregan métricas de reseñas (`review_score`) a nivel de cliente para capturar satisfacción.

In [79]:
orders_order_reviews = orders.merge(order_reviews, on="order_id", how="left")

avg_review = (
    orders_order_reviews.groupby("customer_id")["review_score"]
                  .mean()
                  .reset_index()
)

avg_review.columns = ["customer_id", "avg_review_score"]
avg_review.head()


,customer_id,avg_review_score
0,00012a2ce6f8dcda20d059ce98491703,1.0
1,000161a058600d5901f007fab4c27140,4.0
2,0001fd6190edaaf884bcaf3d49edf079,5.0
3,0002414f95344307404f0ace7a26f1d5,5.0
4,000379cdec625522490c315e70c7a9fb,4.0


## Feature: pagos

Se agregan métricas de pagos a nivel cliente (por ejemplo, cantidad de cuotas o valor).

In [80]:
orders_order_payments = orders.merge(order_payments, on="order_id", how="left")

payment_type = (
    orders_order_payments.groupby("customer_id")["payment_type"]
                   .agg(lambda x: x.mode()[0] if not x.mode().empty else "unknown")
                   .reset_index()
)

payment_type.columns = ["customer_id", "main_payment_type"]
payment_type.head()


,customer_id,main_payment_type
0,00012a2ce6f8dcda20d059ce98491703,credit_card
1,000161a058600d5901f007fab4c27140,credit_card
2,0001fd6190edaaf884bcaf3d49edf079,credit_card
3,0002414f95344307404f0ace7a26f1d5,boleto
4,000379cdec625522490c315e70c7a9fb,boleto


## Ensamble final del dataset de churn

Se integran todas las features construidas a un solo DataFrame a nivel cliente.

In [81]:
churn_features = customers_churn \
    .merge(purchase_frequency, on="customer_id", how="left") \
    .merge(total_spent, on="customer_id", how="left") \
    .merge(avg_ticket, on="customer_id", how="left") \
    .merge(avg_review, on="customer_id", how="left") \
    .merge(payment_type, on="customer_id", how="left")

churn_features.head()


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,last_purchase_date,days_since_last_purchase,churn,purchase_count,total_spent,avg_ticket,avg_review_score,main_payment_type
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,2017-05-16 15:05:35,519,1,1,124.99,124.99,4.0,credit_card
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,2018-01-12 20:48:24,277,1,1,289.00,289.00,5.0,credit_card
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,2018-05-19 16:07:45,151,0,1,139.94,139.94,5.0,credit_card
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,2018-03-13 16:06:38,218,1,1,149.94,149.94,5.0,credit_card
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,2018-07-29 09:51:30,80,0,1,230.00,230.00,5.0,credit_card


## Validación de valores faltantes

Se revisan valores nulos para decidir imputación/limpieza mínima antes del modelado.

In [82]:
churn_features.isna().sum()


customer_id                   0
customer_unique_id            0
customer_zip_code_prefix      0
customer_city                 0
customer_state                0
last_purchase_date            0
days_since_last_purchase      0
churn                         0
purchase_count                0
total_spent                   0
avg_ticket                  775
avg_review_score            768
main_payment_type             0
dtype: int64

## Limpieza e imputación

Se imputan valores faltantes en features numéricas (ej. 0 compras) para evitar problemas en el pipeline.

In [83]:
churn_features["purchase_count"] = churn_features["purchase_count"].fillna(0)
churn_features["total_spent"] = churn_features["total_spent"].fillna(0)
churn_features["avg_ticket"] = churn_features["avg_ticket"].fillna(0)
churn_features["avg_review_score"] = churn_features["avg_review_score"].fillna(churn_features["avg_review_score"].median())
churn_features["main_payment_type"] = churn_features["main_payment_type"].fillna("unknown")


## Separación de variables (X) y objetivo (y)

Se define `X` como las variables predictoras y `y` como el target `churn`.

In [84]:
# Variable objetivo
y = churn_features["churn"]

# Features (sacamos IDs y columnas que no deben entrar al modelo)
X = churn_features.drop(
    columns=[
        "customer_id",
        "churn",
        "last_purchase_date"
    ]
)

X.head()


,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,days_since_last_purchase,purchase_count,total_spent,avg_ticket,avg_review_score,main_payment_type
0,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,519,1,124.99,124.99,4.0,credit_card
1,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,277,1,289.00,289.00,5.0,credit_card
2,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,151,1,139.94,139.94,5.0,credit_card
3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,218,1,149.94,149.94,5.0,credit_card
4,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,80,1,230.00,230.00,5.0,credit_card


## Tipos de variables: numéricas vs categóricas

Se identifican columnas numéricas y categóricas para aplicar transformaciones apropiadas (escalado y one-hot).

In [85]:
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

numeric_features, categorical_features


(Index(['customer_zip_code_prefix', 'days_since_last_purchase',
        'purchase_count', 'total_spent', 'avg_ticket', 'avg_review_score'],
       dtype='object'),
 Index(['customer_unique_id', 'customer_city', 'customer_state',
        'main_payment_type'],
       dtype='object'))

## Pipeline de preprocesamiento (scaling + one-hot)

Se crea un `ColumnTransformer` para:
- escalar variables numéricas
- aplicar One-Hot Encoding a variables categóricas

In [86]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


## Split de entrenamiento y prueba

Se separan datos en train/test para evaluar modelos de forma justa.

In [87]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((79552, 10), (19889, 10))

## Transformación final de X

Se ajusta el preprocesador con `train` y se transforma `train` y `test`.

In [88]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

X_train_processed.shape


(79552, 81275)

## Exportación

Se guardan datasets finales para usar en el notebook de modelado (`03_churn_modeling.ipynb`).

In [89]:
churn_features.to_csv("../data/churn_features.csv", index=False)
churn_features.to_csv("../data/churn_processed.csv", index=False)

